# 🏁 RaceTagger Scene Classifier Training v2 - ResNet18 Optimized

**Obiettivo**: Replicare risultati Roboflow (89% accuracy) con training locale.

## Ottimizzazioni Chiave:
- ✅ **ResNet18** (11.7M params) invece di ResNet50 (25.6M) - previene overfitting
- ✅ **MixUp** augmentation - specifico per classification
- ✅ **Label Smoothing 0.1** - previene overconfidence
- ✅ **Cosine Annealing LR** con warmup
- ✅ **Class Weights** ottimizzati per bilanciamento
- ✅ **Augmentation potenziata** (rotation ±25°, brightness 0.7-1.3)

## Setup
1. Runtime → Change runtime type → **GPU (T4)**
2. Dataset già caricato su Google Drive
3. Run cells sequentially

**Target**: 85-91% validation accuracy

In [ ]:
# Check GPU availability
!nvidia-smi

# Install dependencies
!pip install -q tensorflow pillow tensorflowjs matplotlib seaborn
!pip install -q image-classifiers  # Pacchetto PyPI corretto per ResNet18

import tensorflow as tf
print(f"\n✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")

# Verify classification_models - import corretto
from classification_models.keras import Classifiers
print("✅ classification_models installed")

In [ ]:
# Mount Google Drive
from google.colab import drive
import os
from pathlib import Path

drive.mount('/content/drive')

print("\n🔍 Searching for dataset on Google Drive...\n")

# Possible dataset locations on Drive
POSSIBLE_PATHS = [
    '/content/drive/MyDrive/FP - Federico Pasinetti/Progetti Personali/RaceTagger/scene_classifier_ML/f1_scenes_dataset',
    '/content/drive/MyDrive/f1_scenes_dataset',
    '/content/drive/MyDrive/RaceTagger/f1_scenes_dataset',
    '/content/drive/MyDrive/ml-training/f1_scenes_dataset',
]

# Find dataset
DATASET_ROOT = None
for path in POSSIBLE_PATHS:
    if os.path.exists(path):
        DATASET_ROOT = path
        print(f"✅ Dataset found: {path}")
        break

if DATASET_ROOT is None:
    print("❌ Dataset not found in common locations!")
    print("\n📂 Available folders in MyDrive:")
    !ls -la /content/drive/MyDrive/
    raise FileNotFoundError("Dataset not found on Google Drive")

# Set processed dataset path
DATASET_PATH = os.path.join(DATASET_ROOT, 'processed')

# Verify structure
required_dirs = ['train', 'val', 'test']
for subdir in required_dirs:
    subdir_path = os.path.join(DATASET_PATH, subdir)
    if not os.path.exists(subdir_path):
        raise FileNotFoundError(f"Expected directory not found: {subdir_path}")
    categories = os.listdir(subdir_path)
    print(f"✅ {subdir:5s}: {len(categories)} categories")

print(f"\n✅ Dataset ready: {DATASET_PATH}")

In [ ]:
# Configuration - OPTIMIZED for Classification
import os
import json
import numpy as np
from pathlib import Path

# Training config
INPUT_SIZE = (224, 224)
BATCH_SIZE = 32
RANDOM_SEED = 42

# Categories
CATEGORIES = [
    'crowd_scene',
    'garage_pitlane',
    'podium_celebration',
    'portrait_paddock',
    'racing_action'
]
NUM_CLASSES = len(CATEGORIES)

# ResNet18 OPTIMIZED config
CONFIG = {
    # Phase 1: Train classification head only
    'phase1_epochs': 15,
    'phase1_lr': 5e-4,      # Lower than before (was 1e-3)
    
    # Phase 2: Fine-tune with gradual unfreezing
    'phase2_epochs': 35,
    'phase2_lr': 1e-4,
    'unfreeze_layers': 30,  # ResNet18 has fewer layers
    
    # Architecture
    'dense_units': 256,
    'dropout': 0.4,         # Increased from 0.3
    
    # Regularization
    'label_smoothing': 0.1,
    'mixup_alpha': 0.2,
    
    # Early stopping
    'patience': 10,         # Increased from 5
}

# Data augmentation - ENHANCED
AUGMENTATION = {
    'rotation_range': 25,           # Increased from 15
    'width_shift_range': 0.15,      # Increased from 0.1
    'height_shift_range': 0.15,     # Increased from 0.1
    'zoom_range': 0.2,              # Increased from 0.15
    'brightness_range': (0.7, 1.3), # Increased from (0.8, 1.2)
    'shear_range': 0.15,            # NEW
    'channel_shift_range': 20.0,    # NEW
    'horizontal_flip': True,
    'fill_mode': 'reflect',         # Changed from 'nearest'
}

print("✅ Configuration loaded")
print(f"\n📋 Key settings:")
print(f"   Model: ResNet18 (11.7M params)")
print(f"   Label smoothing: {CONFIG['label_smoothing']}")
print(f"   MixUp alpha: {CONFIG['mixup_alpha']}")
print(f"   Dropout: {CONFIG['dropout']}")
print(f"   Phase 1 LR: {CONFIG['phase1_lr']}")
print(f"   Phase 2 LR: {CONFIG['phase2_lr']}")

In [ ]:
# Data loading with MixUp augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from typing import Tuple, Dict, Generator
import numpy as np

def compute_class_weights(train_dir: Path) -> Dict[int, float]:
    """Compute optimized class weights for imbalanced datasets"""
    class_counts = {}
    
    for i, category in enumerate(sorted(CATEGORIES)):
        category_dir = train_dir / category
        if category_dir.exists():
            n_images = len(list(category_dir.glob('*.jpg')))
            class_counts[i] = n_images
    
    total_images = sum(class_counts.values())
    n_classes = len(class_counts)
    
    # Effective number weighting (better for extreme imbalance)
    beta = (total_images - 1) / total_images
    class_weights = {
        i: (1 - beta) / (1 - beta ** count)
        for i, count in class_counts.items()
    }
    
    # Normalize to prevent huge values
    max_weight = max(class_weights.values())
    class_weights = {k: v / max_weight * 2 for k, v in class_weights.items()}
    
    return class_weights

def mixup_generator(generator, alpha=0.2):
    """
    MixUp augmentation for classification.
    Blends images and labels for better generalization.
    """
    while True:
        # Get two batches
        x1, y1 = next(generator)
        x2, y2 = next(generator)
        
        # Ensure same batch size
        batch_size = min(len(x1), len(x2))
        x1, y1 = x1[:batch_size], y1[:batch_size]
        x2, y2 = x2[:batch_size], y2[:batch_size]
        
        # MixUp: blend images and labels
        lam = np.random.beta(alpha, alpha, batch_size).reshape(-1, 1, 1, 1)
        lam_y = lam.reshape(-1, 1)
        
        x_mixed = lam * x1 + (1 - lam) * x2
        y_mixed = lam_y * y1 + (1 - lam_y) * y2
        
        yield x_mixed, y_mixed

def create_data_generators():
    """Create training and validation data generators with ENHANCED augmentation"""
    
    # Get preprocessing function for ResNet - IMPORT CORRETTO
    from classification_models.keras import Classifiers
    _, preprocess_input = Classifiers.get('resnet18')
    
    # Training augmentation - ENHANCED
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=AUGMENTATION['rotation_range'],
        width_shift_range=AUGMENTATION['width_shift_range'],
        height_shift_range=AUGMENTATION['height_shift_range'],
        zoom_range=AUGMENTATION['zoom_range'],
        brightness_range=AUGMENTATION['brightness_range'],
        shear_range=AUGMENTATION['shear_range'],
        channel_shift_range=AUGMENTATION['channel_shift_range'],
        horizontal_flip=AUGMENTATION['horizontal_flip'],
        fill_mode=AUGMENTATION['fill_mode']
    )
    
    # Validation (only preprocessing, no augmentation)
    val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input
    )
    
    return train_datagen, val_datagen

def load_datasets(train_datagen, val_datagen, dataset_path):
    """Load train and validation datasets"""
    train_path = Path(dataset_path) / 'train'
    val_path = Path(dataset_path) / 'val'
    
    # Training set
    train_generator = train_datagen.flow_from_directory(
        str(train_path),
        target_size=INPUT_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True,
        seed=RANDOM_SEED
    )
    
    # Validation set
    val_generator = val_datagen.flow_from_directory(
        str(val_path),
        target_size=INPUT_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator

# Load data
train_datagen, val_datagen = create_data_generators()
train_gen, val_gen = load_datasets(train_datagen, val_datagen, DATASET_PATH)

# Compute class weights
class_weights = compute_class_weights(Path(DATASET_PATH) / 'train')

print(f"\n📊 Dataset Info:")
print(f"  Train samples: {train_gen.samples}")
print(f"  Val samples: {val_gen.samples}")
print(f"  Classes: {CATEGORIES}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"\n⚖️  Class weights (normalized):")
for i, cat in enumerate(sorted(CATEGORIES)):
    print(f"   {cat}: {class_weights[i]:.3f}")

In [ ]:
# Model building - ResNet18 with optimized head
from tensorflow import keras
from tensorflow.keras import layers
from classification_models.keras import Classifiers  # IMPORT CORRETTO

def build_resnet18_model(num_classes: int, freeze_base: bool = True):
    """
    Build ResNet18 with optimized classification head.
    
    Key improvements:
    - ResNet18 (11.7M params) instead of ResNet50 (25.6M) - prevents overfitting
    - BatchNorm in dense head for stability
    - Higher dropout (0.4) for regularization
    """
    print("\n🏗️  Building ResNet18 Classification Model...")
    
    # Get ResNet18 from classification_models library
    ResNet18, _ = Classifiers.get('resnet18')
    
    base_model = ResNet18(
        input_shape=(*INPUT_SIZE, 3),
        weights='imagenet',
        include_top=False
    )
    
    base_model.trainable = not freeze_base
    
    print(f"  Base model: ResNet18")
    print(f"  Base parameters: {base_model.count_params():,}")
    print(f"  Trainable: {not freeze_base}")
    print(f"  Total base layers: {len(base_model.layers)}")
    
    # Build optimized classification head
    inputs = keras.Input(shape=(*INPUT_SIZE, 3))
    
    # Base model (let Keras handle training mode automatically)
    x = base_model(inputs)
    
    # Global pooling
    x = layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    
    # Dense block with BatchNorm (stabilizes training)
    x = layers.Dense(CONFIG['dense_units'], activation=None, name='dense_classifier')(x)
    x = layers.BatchNormalization(name='bn_dense')(x)
    x = layers.Activation('relu', name='relu_dense')(x)
    
    # Dropout (increased for small dataset)
    x = layers.Dropout(CONFIG['dropout'], name='dropout')(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
    
    model = keras.Model(inputs, outputs, name='resnet18_scene_classifier')
    
    print(f"\n  Model architecture:")
    print(f"    Input: {INPUT_SIZE[0]}x{INPUT_SIZE[1]}x3")
    print(f"    Base: ResNet18 (ImageNet pretrained)")
    print(f"    Pool: GlobalAveragePooling2D")
    print(f"    Dense: {CONFIG['dense_units']} units + BatchNorm + ReLU")
    print(f"    Dropout: {CONFIG['dropout']}")
    print(f"    Output: {num_classes} classes (Softmax)")
    print(f"\n  Total parameters: {model.count_params():,}")
    
    return model, base_model

def unfreeze_top_layers(base_model, n_layers: int):
    """Unfreeze top N layers of base model for fine-tuning"""
    base_model.trainable = True
    
    # Freeze all layers except top N
    for layer in base_model.layers[:-n_layers]:
        layer.trainable = False
    
    trainable_count = sum([1 for layer in base_model.layers if layer.trainable])
    print(f"\n🔓 Unfrozen top {trainable_count}/{len(base_model.layers)} layers")

print("✅ Model builder ready")

In [ ]:
# Training utilities with Cosine Annealing LR
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    ReduceLROnPlateau,
    LearningRateScheduler,
    Callback
)
from datetime import datetime
import math

def cosine_annealing_schedule(epoch, initial_lr, max_epochs, warmup_epochs=3):
    """
    Cosine annealing learning rate schedule with warmup.
    
    - Warmup: Linear increase for first N epochs
    - Cosine: Smooth decay following cosine curve
    """
    if epoch < warmup_epochs:
        # Linear warmup
        return initial_lr * (epoch + 1) / warmup_epochs
    else:
        # Cosine annealing
        progress = (epoch - warmup_epochs) / (max_epochs - warmup_epochs)
        return initial_lr * 0.5 * (1 + math.cos(math.pi * progress))

def create_callbacks(phase: str):
    """Create training callbacks with cosine LR scheduling"""
    callbacks = []
    
    # Model checkpoint
    checkpoint_dir = Path('/content/checkpoints/resnet18')
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    checkpoint_cb = ModelCheckpoint(
        str(checkpoint_dir / f'{phase}_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    callbacks.append(checkpoint_cb)
    
    # Early stopping with increased patience
    early_stop_cb = EarlyStopping(
        monitor='val_loss',
        patience=CONFIG['patience'],
        restore_best_weights=True,
        verbose=1
    )
    callbacks.append(early_stop_cb)
    
    # Cosine annealing LR schedule
    if phase == 'phase1':
        max_epochs = CONFIG['phase1_epochs']
        initial_lr = CONFIG['phase1_lr']
    else:
        max_epochs = CONFIG['phase2_epochs']
        initial_lr = CONFIG['phase2_lr']
    
    lr_schedule = LearningRateScheduler(
        lambda epoch: cosine_annealing_schedule(epoch, initial_lr, max_epochs),
        verbose=1
    )
    callbacks.append(lr_schedule)
    
    return callbacks

print("✅ Training utilities ready")

## 🚀 Train ResNet18 Scene Classifier

**Two-phase transfer learning:**
1. **Phase 1**: Train classification head (base frozen)
2. **Phase 2**: Fine-tune top layers with lower LR

**Key optimizations:**
- ResNet18 (vs ResNet50) - prevents overfitting
- Label smoothing 0.1
- MixUp augmentation
- Cosine annealing LR
- Class weights for imbalance

**Target**: 85-91% validation accuracy

In [ ]:
# Build model
model, base_model = build_resnet18_model(
    num_classes=NUM_CLASSES,
    freeze_base=True
)

# ─────────────────────────────────────────────
# PHASE 1: Train Classification Head
# ─────────────────────────────────────────────
print("\n" + "="*60)
print("🔒 PHASE 1: Train Classification Head")
print("="*60)

# Compile with label smoothing
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=CONFIG['phase1_lr']),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=CONFIG['label_smoothing']),
    metrics=['accuracy']
)

# Create MixUp generator for training
train_gen_mixup = mixup_generator(train_gen, alpha=CONFIG['mixup_alpha'])

# Calculate steps per epoch
steps_per_epoch = train_gen.samples // BATCH_SIZE

# Phase 1 callbacks
phase1_callbacks = create_callbacks('phase1')

# Train Phase 1 (class_weight rimosso - MixUp + Label Smoothing gestiscono lo sbilanciamento)
history_phase1 = model.fit(
    train_gen_mixup,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    epochs=CONFIG['phase1_epochs'],
    callbacks=phase1_callbacks,
    verbose=1
)

print(f"\n✅ Phase 1 Complete")
print(f"   Best Val Accuracy: {max(history_phase1.history['val_accuracy']):.4f}")

In [ ]:
# ─────────────────────────────────────────────
# PHASE 2: Fine-tune Top Layers
# ─────────────────────────────────────────────
print("\n" + "="*60)
print("🔓 PHASE 2: Fine-tune Top Layers")
print("="*60)

# Unfreeze top layers
unfreeze_top_layers(base_model, CONFIG['unfreeze_layers'])

# Recompile with lower LR for fine-tuning
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=CONFIG['phase2_lr']),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=CONFIG['label_smoothing']),
    metrics=['accuracy']
)

# Recreate MixUp generator (reset)
train_gen_mixup = mixup_generator(train_gen, alpha=CONFIG['mixup_alpha'])

# Phase 2 callbacks
phase2_callbacks = create_callbacks('phase2')

# Train Phase 2 (class_weight rimosso - MixUp + Label Smoothing gestiscono lo sbilanciamento)
history_phase2 = model.fit(
    train_gen_mixup,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    epochs=CONFIG['phase2_epochs'],
    callbacks=phase2_callbacks,
    verbose=1
)

# Save final model
model.save('/content/resnet18_scene_classifier_final.keras')

print(f"\n✅ Phase 2 Complete")
print(f"   Best Val Accuracy: {max(history_phase2.history['val_accuracy']):.4f}")

In [ ]:
# Training results summary
import matplotlib.pyplot as plt

# Combine histories
all_accuracy = history_phase1.history['accuracy'] + history_phase2.history['accuracy']
all_val_accuracy = history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy']
all_loss = history_phase1.history['loss'] + history_phase2.history['loss']
all_val_loss = history_phase1.history['val_loss'] + history_phase2.history['val_loss']

# Final metrics
final_val_accuracy = max(all_val_accuracy)
final_val_loss = min(all_val_loss)

print("\n" + "="*60)
print("📊 TRAINING RESULTS")
print("="*60)
print(f"\n🏆 BEST VALIDATION ACCURACY: {final_val_accuracy:.4f}")
print(f"   Best Validation Loss: {final_val_loss:.4f}")
print(f"\n   Phase 1 best: {max(history_phase1.history['val_accuracy']):.4f}")
print(f"   Phase 2 best: {max(history_phase2.history['val_accuracy']):.4f}")

# Check target
target_accuracy = 0.85
if final_val_accuracy >= target_accuracy:
    print(f"\n✅ TARGET REACHED! ({final_val_accuracy:.1%} >= {target_accuracy:.0%})")
else:
    print(f"\n⚠️  Target not reached ({final_val_accuracy:.1%} < {target_accuracy:.0%})")
    print(f"   Gap: {(target_accuracy - final_val_accuracy)*100:.1f}%")

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
ax1.plot(all_accuracy, label='Train')
ax1.plot(all_val_accuracy, label='Validation')
ax1.axvline(x=len(history_phase1.history['accuracy'])-1, color='r', linestyle='--', alpha=0.5, label='Phase 1→2')
ax1.axhline(y=target_accuracy, color='g', linestyle='--', alpha=0.5, label=f'Target ({target_accuracy:.0%})')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.set_ylim(0, 1)

# Loss
ax2.plot(all_loss, label='Train')
ax2.plot(all_val_loss, label='Validation')
ax2.axvline(x=len(history_phase1.history['loss'])-1, color='r', linestyle='--', alpha=0.5, label='Phase 1→2')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.tight_layout()
plt.savefig('/content/training_history.png', dpi=150)
plt.show()

print("\n✅ Training history saved: /content/training_history.png")

## 📦 Export to TensorFlow.js

Convert trained model to TensorFlow.js format for Electron deployment.

In [ ]:
import tensorflowjs as tfjs

# Load best model
best_model = keras.models.load_model('/content/resnet18_scene_classifier_final.keras')

# Export to TensorFlow.js
tfjs_output_dir = '/content/tfjs_models/resnet18'
tfjs.converters.save_keras_model(best_model, tfjs_output_dir)

print(f"\n✅ Model exported to: {tfjs_output_dir}")

# Export quantized version (int8) for smaller size
tfjs_quantized_dir = '/content/tfjs_models/resnet18_quantized'
tfjs.converters.save_keras_model(
    best_model,
    tfjs_quantized_dir,
    quantization_dtype_map={'uint8': '*'}
)

print(f"✅ Quantized model exported to: {tfjs_quantized_dir}")

# Save class labels and config
model_info = {
    'categories': CATEGORIES,
    'category_to_index': {cat: i for i, cat in enumerate(CATEGORIES)},
    'index_to_category': {i: cat for i, cat in enumerate(CATEGORIES)},
    'input_size': INPUT_SIZE,
    'num_classes': NUM_CLASSES,
    'model_type': 'ResNet18',
    'preprocessing': 'classification_models resnet18 preprocess_input',
    'final_val_accuracy': float(final_val_accuracy),
    'training_config': CONFIG
}

with open(f'{tfjs_output_dir}/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

with open(f'{tfjs_quantized_dir}/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

# Create zip for download
!zip -r /content/scene_classifier_resnet18_tfjs.zip /content/tfjs_models/

print("\n✅ Export complete!")
print(f"\n📦 Download: /content/scene_classifier_resnet18_tfjs.zip")

In [ ]:
# Test model on sample images
from tensorflow.keras.preprocessing import image
from classification_models.keras import Classifiers  # IMPORT CORRETTO
import numpy as np

# Get preprocessing function
_, preprocess_input = Classifiers.get('resnet18')

# Load test images
test_dir = Path(DATASET_PATH) / 'test'

print("\n🧪 Testing model on sample images...\n")

correct = 0
total = 0

for category in CATEGORIES:
    category_dir = test_dir / category
    if not category_dir.exists():
        continue
    
    test_images = list(category_dir.glob('*.jpg'))[:5]  # Test 5 images per category
    
    for img_path in test_images:
        # Load and preprocess image
        img = image.load_img(img_path, target_size=INPUT_SIZE)
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        
        # Predict
        predictions = best_model.predict(img_array, verbose=0)
        predicted_class = CATEGORIES[np.argmax(predictions[0])]
        confidence = np.max(predictions[0])
        
        # Check if correct
        is_correct = predicted_class == category
        if is_correct:
            correct += 1
        total += 1
        
        status = "✅" if is_correct else "❌"
        print(f"{status} True: {category:20s} | Pred: {predicted_class:20s} | Conf: {confidence:.2%}")

test_accuracy = correct / total if total > 0 else 0
print(f"\n📊 Test Accuracy: {test_accuracy:.2%} ({correct}/{total})")

In [ ]:
from google.colab import files

print("\n📥 Downloading trained models...\n")

# Save training summary
training_summary = {
    'model': 'ResNet18',
    'final_val_accuracy': float(final_val_accuracy),
    'final_val_loss': float(final_val_loss),
    'phase1_best_acc': float(max(history_phase1.history['val_accuracy'])),
    'phase2_best_acc': float(max(history_phase2.history['val_accuracy'])),
    'total_epochs': len(all_accuracy),
    'parameters': int(model.count_params()),
    'config': CONFIG,
    'dataset_info': {
        'train_samples': train_gen.samples,
        'val_samples': val_gen.samples,
        'categories': CATEGORIES
    }
}

with open('/content/training_summary.json', 'w') as f:
    json.dump(training_summary, f, indent=2)

# Download files
print("1. TensorFlow.js models (all formats)")
files.download('/content/scene_classifier_resnet18_tfjs.zip')

print("2. Keras model")
files.download('/content/resnet18_scene_classifier_final.keras')

print("3. Training summary")
files.download('/content/training_summary.json')

print("4. Training history plot")
files.download('/content/training_history.png')

print("\n✅ All downloads complete!")
print("\n📋 Next steps:")
print("1. Extract scene_classifier_resnet18_tfjs.zip")
print("2. Copy resnet18_quantized/ to RaceTagger project")
print("3. Test inference in Electron app")
print("4. Integrate preprocessing (use classification_models preprocess_input)")